In [1]:
import numpy as np
import plotly.graph_objects as go
from scipy.optimize import minimize
from tqdm import tqdm
from scipy.stats import t, multivariate_normal, multivariate_t
from scipy.special import logsumexp, gamma, gammaln
from numpy.linalg import inv, det, slogdet, LinAlgError
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from numpy import tril_indices, triu_indices
from typing import Literal, Optional, Dict, Union

from Functions.simulation_1 import * 
from Functions.gas_filter_5 import *
from Functions.kf_filter_5 import *

In [2]:
from xbbg import blp
import pandas as pd

# Univariate DCS-t Filter: Appendix (For Demo)

In [3]:
list_to_pull = ["FWISUS55 Index", "USGG10YR Index", "CO1 Comdty"]
colnames = ['ILS5Y5Y', 'US10Y', 'Brent']
today = pd.Timestamp.today().strftime('%Y-%m-%d') 
# yesterday = (pd.Timestamp.today() - pd.Timedelta(days=1)).strftime('%Y-%m-%d')
data = blp.bdh(
    tickers=list_to_pull, 
    flds=['PX_LAST'], 
    start_date='2000-01-01',
    end_date=today)

data.columns = data.columns.get_level_values(0)
data.index = pd.to_datetime(data.index)
data.columns = colnames


data_1 = data.ffill().bfill()['2019-01-01':].resample('W-FRI').last()

In [ ]:
# data.to_csv("Empirical_Application_Univariate.csv")

In [4]:
y_univariate = data_1['ILS5Y5Y'].values.reshape(-1, 1)[1:]
X_univariate = np.column_stack((np.ones(len(data_1)), data_1['US10Y'].diff(), np.log(data_1['Brent']).diff()))[1:]

In [5]:
## Running the univariate Kalman filter
kf_results_univariate = multivariate_KF_with_estimation(
    y = y_univariate,
    X = X_univariate,   
)

mu_kf_univariate = kf_results_univariate['mu_filtered']
beta_kf_univariate = kf_results_univariate['beta']
y_hat_kf_univariate = mu_kf_univariate + X_univariate @ beta_kf_univariate


✅ Kalman estimation completed successfully.
Estimated parameters:
omega (unconditional mean): [0.1053899]
Phi (persistence matrix): 
[[0.97361908]]
beta: 
[[2.29791312]
 [0.07603174]
 [0.11548344]]
Q (state noise covariance): 
[[0.003477]]
R (observation noise covariance): 
[[0.00116482]]
Log-likelihood: 400.82997414722706
AIC: -787.6599482944541
BIC: -760.7754566928403


In [6]:
dcs_results_univariate = estimate_and_filter_gas(
    y = y_univariate,
    X = X_univariate,
    fix_nu= None,         # or None to estimate
)

mu_dcs_univariate = dcs_results_univariate['mu_filtered']
beta_dcs_univariate = dcs_results_univariate['beta']
y_hat_dcs_univariate = mu_dcs_univariate + X_univariate @ beta_dcs_univariate

Estimated omega:
 [0.05959877]
Estimated Phi:
 [[0.995]]
Estimated beta:
 [[ 2.13063426e+00]
 [ 1.01604342e-01]
 [-5.60646684e-04]]
Estimated Omega (from lambda):
 [[0.00334059]]
Estimated Kappa:
 [[0.00364788]]
Estimated nu:
 6.788276596362299
Log-likelihood: 440.1721362710807
AIC: -864.3442725421614
BIC: -833.6191392831743


In [7]:
dcs_results_univariate_fix5 = estimate_and_filter_gas(
    y = y_univariate,
    X = X_univariate,
    fix_nu= 5,         # or None to estimate
)

dcs_results_univariate_fix10 = estimate_and_filter_gas(
    y = y_univariate,
    X = X_univariate,
    fix_nu= 10,         # or None to estimate
)

dcs_results_univariate_fix10 = estimate_and_filter_gas(
    y = y_univariate,
    X = X_univariate,
    fix_nu= 20,         # or None to estimate
)

Estimated omega:
 [-0.43442397]
Estimated Phi:
 [[0.995]]
Estimated beta:
 [[2.62335391]
 [0.09456257]
 [0.01153324]]
Estimated Omega (from lambda):
 [[0.00312448]]
Estimated Kappa:
 [[0.00365912]]
Estimated nu:
 5
Log-likelihood: 439.27706038553464
AIC: -864.5541207710693
BIC: -837.6696291694554
Estimated omega:
 [-0.50383942]
Estimated Phi:
 [[0.99128973]]
Estimated beta:
 [[2.70109157e+00]
 [1.05460288e-01]
 [1.00256053e-03]]
Estimated Omega (from lambda):
 [[0.00357634]]
Estimated Kappa:
 [[0.00361463]]
Estimated nu:
 10
Log-likelihood: 438.9147731026417
AIC: -863.8295462052833
BIC: -836.9450546036695
Estimated omega:
 [0.26760665]
Estimated Phi:
 [[0.98466842]]
Estimated beta:
 [[1.94343534]
 [0.10235748]
 [0.03875681]]
Estimated Omega (from lambda):
 [[0.00400698]]
Estimated Kappa:
 [[0.00373407]]
Estimated nu:
 20
Log-likelihood: 432.51514143477243
AIC: -851.0302828695449
BIC: -824.145791267931


In [8]:
T_univariate, R_univariate =  y_univariate.shape
time_axis = data_1.index #list(range(T_univariate))

# --- Residuals ---
residuals_dcs = y_univariate - y_hat_dcs_univariate
residuals_kf = y_univariate - y_hat_kf_univariate

# --- Define subplot layout: R rows for dimensions, 1 extra for residuals
total_rows = R_univariate + 1
row_heights = [0.7 / R_univariate] * R_univariate + [0.3]  # Normalize: R plots get 70%, residuals get 30%

fig_univariate = make_subplots(
    rows=total_rows, cols=1,
    shared_xaxes=True,
    subplot_titles=[f"Dimension {i+1}" for i in range(R_univariate)] + ["Residuals: uDCS-t vs Kalman"],
    row_heights=row_heights
)

# --- Add observed & filtered traces ---
for i in range(R_univariate):
    fig_univariate.add_trace(go.Scatter(x=time_axis, y=y_univariate[:, i],
        mode='lines', name=f'y[{i+1}] (Observed)',
        line=dict(color='gray', width=1.5, dash='dot')), row=i+1, col=1)

    fig_univariate.add_trace(go.Scatter(x=time_axis, y=y_hat_dcs_univariate[:, i],
        mode='lines', name=f'y_hat_dcs[{i+1}]',
        line=dict(color='orange', width=2, dash='dash')), row=i+1, col=1)

    fig_univariate.add_trace(go.Scatter(x=time_axis, y=y_hat_kf_univariate[:, i],
        mode='lines', name=f'y_hat_kf[{i+1}]',
        line=dict(color='blue', width=2, dash='dash')), row=i+1, col=1)

    fig_univariate.add_trace(go.Scatter(x=time_axis, y=mu_dcs_univariate[:, i],
        mode='lines', name=f'mu_dcs[{i+1}]',
        line=dict(color='red', width=2, dash='dash')), row=i+1, col=1)

    fig_univariate.add_trace(go.Scatter(x=time_axis, y=mu_kf_univariate[:, i],
        mode='lines', name=f'mu_kf[{i+1}]',
        line=dict(color='green', width=2, dash='dash')), row=i+1, col=1)

# --- Add residual comparison plot ---
fig_univariate.add_trace(go.Scatter(
    x=time_axis, y=residuals_dcs[:, 0], name="Residual (dcs)",
    line=dict(color='red', dash='solid')), row=total_rows, col=1)

fig_univariate.add_trace(go.Scatter(
    x=time_axis, y=residuals_kf[:, 0], name="Residual (Kalman)",
    line=dict(color='blue', dash='solid')), row=total_rows, col=1)

# --- Final layout ---
fig_univariate.update_layout(
    height=300*R_univariate + 200, width=950,
    title_text="DCS-t Filter: True vs Filtered Latent States",
    template="plotly_white",
    showlegend=True
)

fig_univariate.update_xaxes(title_text="Time", row=total_rows, col=1)
fig_univariate.update_yaxes(title_text="Value", row=total_rows, col=1)

fig_univariate.show()

# Multivariate Models

## ILS5Y5Y: US & EU

In [35]:
list_to_pull_2 = ["FWISUS55 Index", "FWISEU55 Index", "USGG10YR Index", "GDBR10 Index",  "CO1 Comdty", 'SPX Index', "DAX Index", "SX5E Index", "DXY Index"]
colnames_2 = ['US ILS5Y5Y', "EU ILS5Y5Y", 'US10Y', "DE10Y", 'Brent', "SPX", "DAX", "EuroStoxx50", "DXY"]


today = pd.Timestamp.today().strftime('%Y-%m-%d') 
# yesterday = (pd.Timestamp.today() - pd.Timedelta(days=1)).strftime('%Y-%m-%d')
data_2 = blp.bdh(
    tickers=list_to_pull_2, 
    flds=['PX_LAST'], 
    start_date='2000-01-01',
    end_date=today)

data_2.columns = data_2.columns.get_level_values(0)
data_2.index = pd.to_datetime(data_2.index)
data_2.columns = colnames_2

In [ ]:
# data_2.to_csv("Empirical_Application_Multivariate_1.csv")

In [ ]:
data_2_weekly = data_2.ffill().bfill()['2012-01-01':].resample('W-FRI').last()
df = data_2.resample('W-FRI').last()['2019-01-01':]


df["dUS10Y"] = df["US10Y"].diff()
df["dDE10Y"] = df["DE10Y"].diff()

df["dBrent"] = np.log(df["Brent"]).diff()
df['dSPX'] = np.log(df['SPX']).diff()
df['dDAX'] = np.log(df['DAX']).diff()
df['dEuroStoxx50'] = np.log(df['EuroStoxx50']).diff()

# Drop the first row 
df = df.iloc[1:]
y_multivariate = df[['US ILS5Y5Y', "EU ILS5Y5Y"]].values

# Region specific X
X_us = np.column_stack([  
    df["US10Y"].values,        
    df["dUS10Y"].values,       
    df["dBrent"].values,         
    df['dSPX'].values            
])

X_eu = np.column_stack([
    df['DE10Y'].values,        
    df["dDE10Y"].values, 
    df["dBrent"].values,        
    df['dEuroStoxx50'].values   
])


X_multivariate = [X_us, X_eu]
P_list = [X_us.shape[1], X_eu.shape[1]]  

In [97]:
dcs_results_mult = estimate_and_filter_gas(
    y=y_multivariate,
    X=X_multivariate,
    X_shared=False,
    P_list=P_list,
    phi_type='upper',
    kappa_type='upper',
    fix_nu= 10,         # or None to estimate
    verbose=True,
)

Estimated omega:
 [1.70950722 1.50888648]
Estimated Phi:
 [[ 0.99236867 -0.00916091]
 [ 0.          0.99358078]]
Estimated beta:
 [array([ 0.17933371,  0.02970997, -0.02340312,  0.44239455]), array([0.21159694, 0.00358128, 0.01767424, 0.06258629])]
Estimated Omega (from lambda):
 [[0.00289697 0.        ]
 [0.         0.00214978]]
Estimated Kappa:
 [[0.00234617 0.00062736]
 [0.         0.00294844]]
Estimated nu:
 10
Log-likelihood: 998.9245253004947
AIC: -1961.8490506009894
BIC: -1880.2408515181892


In [98]:
dcs_results_mult_2 = estimate_and_filter_gas(
    y=y_multivariate,
    X=X_multivariate,
    X_shared=False,
    P_list=P_list,
    phi_type='diagonal',
    kappa_type='upper',
    fix_nu= 10,         # or None to estimate
    verbose=True,
)

Estimated omega:
 [1.75170474 1.49881565]
Estimated Phi:
 [[0.98428581 0.        ]
 [0.         0.99494673]]
Estimated beta:
 [array([ 0.16833717,  0.03552805, -0.01888311,  0.42688355]), array([0.20983841, 0.00501287, 0.01629133, 0.06033375])]
Estimated Omega (from lambda):
 [[0.00291087 0.        ]
 [0.         0.00214895]]
Estimated Kappa:
 [[0.0023739  0.00063887]
 [0.         0.00295291]]
Estimated nu:
 10
Log-likelihood: 998.4833894706735
AIC: -1962.966778941347
BIC: -1885.8923686964802


In [99]:
dcs_results_mult_3 = estimate_and_filter_gas(
    y=y_multivariate,
    X=X_multivariate,
    X_shared=False,
    P_list=P_list,
    phi_type='diagonal',
    kappa_type='full',
    fix_nu= 10,         # or None to estimate
    verbose=True,
)

Estimated omega:
 [1.76319876 1.49745592]
Estimated Phi:
 [[0.98257741 0.        ]
 [0.         0.99419594]]
Estimated beta:
 [array([ 0.16645438,  0.03590043, -0.03700463,  0.4443412 ]), array([0.21404333, 0.00467824, 0.02133664, 0.06051151])]
Estimated Omega (from lambda):
 [[0.00291149 0.        ]
 [0.         0.00214553]]
Estimated Kappa:
 [[0.00234466 0.00070399]
 [0.00042321 0.00281551]]
Estimated nu:
 10
Log-likelihood: 1000.679911847825
AIC: -1965.35982369565
BIC: -1883.7516246128498


In [100]:
mu_dcs_multivariate = dcs_results_mult['mu_filtered']
beta_dcs_multivariate = dcs_results_mult['beta']

N = y_multivariate.shape[1]
y_hat_dcs_multivariate = np.zeros_like(mu_dcs_multivariate)
for j in range(N):
    y_hat_dcs_multivariate[:, j] = mu_dcs_multivariate[:, j] + X_multivariate[j] @ dcs_results_mult['beta'][j]

mu_dcs_multivariate_3 = dcs_results_mult_3['mu_filtered']
beta_dcs_multivariate_3 = dcs_results_mult_3['beta']

N = y_multivariate.shape[1]
y_hat_dcs_multivariate_3 = np.zeros_like(mu_dcs_multivariate_3)
for j in range(N):
    y_hat_dcs_multivariate_3[:, j] = mu_dcs_multivariate_3[:, j] + X_multivariate[j] @ dcs_results_mult_3['beta'][j]

In [101]:
kf_results_mult = multivariate_KF_with_estimation(
    y = y_multivariate,
    X = X_multivariate,
    X_shared=False
)

✅ Kalman estimation completed successfully.
Estimated parameters:
omega (unconditional mean): [2.42456894 2.45090711]
Phi (persistence matrix): 
[[3.1909618  0.11063478]
 [0.01245292 3.45399143]]
beta[0]: [0.09480458 0.0135677  0.05178011 0.49967247]
beta[1]: [ 0.15234268 -0.00789068  0.11969287 -0.02812227]
Q (state noise covariance): 
[[0.00317507 0.00188784]
 [0.00188784 0.00234194]]
R (observation noise covariance): 
[[9.74734060e-04 3.12047122e-05]
 [3.12047122e-05 5.00328584e-04]]
Log-likelihood: 971.0146408445333
AIC: -1902.0292816890667
BIC: -1811.3535049303998


In [103]:
mu_kf_multivariate = kf_results_mult['mu_filtered']
beta_kf_multivariate = kf_results_mult['beta']

N = y_multivariate.shape[1]
y_hat_kf_multivariate = np.zeros_like(mu_kf_multivariate)
for j in range(N):
    y_hat_kf_multivariate[:, j] = mu_kf_multivariate[:, j] + X_multivariate[j] @ kf_results_mult['beta'][j]

In [ ]:
T_multivariate, R_multivariate =  y_multivariate.shape
time_axis = df.index #list(range(T_multivariate))

# --- Residuals ---
residuals_dcs = y_multivariate - y_hat_dcs_multivariate
residuals_dcs_3 = y_multivariate - y_hat_dcs_multivariate_3
residuals_kf = y_multivariate - y_hat_kf_multivariate


total_rows = 2 * R_multivariate
row_heights = [0.5 / R_multivariate] * R_multivariate + [0.5 / R_multivariate] * R_multivariate


fig_multivariate = make_subplots(
    rows=total_rows, cols=1,
    shared_xaxes=True,
    subplot_titles=[f"Dimension {i+1}" for i in range(R_multivariate)] + ["Residuals: DCS vs Kalman"],
    row_heights=row_heights
)

# Plot Observed and Filtered 
for i in range(R_multivariate):
    fig_multivariate.add_trace(go.Scatter(x=time_axis, y=y_multivariate[:, i],
        mode='lines', name=f'y[{i+1}] (Observed)',
        line=dict(color='gray', width=1.5, dash='dot')), row=i+1, col=1)

    fig_multivariate.add_trace(go.Scatter(x=time_axis, y=y_hat_dcs_multivariate[:, i],
        mode='lines', name=f'y_hat_dcs[{i+1}]',
        line=dict(color='orange', width=2, dash='dash')), row=i+1, col=1)

    fig_multivariate.add_trace(go.Scatter(x=time_axis, y=y_hat_kf_multivariate[:, i],
        mode='lines', name=f'y_hat_kf[{i+1}]',
        line=dict(color='blue', width=2, dash='dash')), row=i+1, col=1)


# Plot the residuals
for i in range(R_multivariate):

    fig_multivariate.add_trace(go.Scatter(
        x=time_axis, y=residuals_dcs[:, i], name=f"Residual (DCS) [{i+1}]",
        line=dict(color='red')), row=i + 1 + R_multivariate, col=1)
    

    fig_multivariate.add_trace(go.Scatter(
        x=time_axis, y=residuals_kf[:, i], name=f"Residual (Kalman) [{i+1}]",
        line=dict(color='blue', dash='dot')), row=i + 1 + R_multivariate, col=1)


fig_multivariate.update_layout(
    height=300*R_multivariate + 200, width=950,
    title_text="mDCS-t Filter: Filtered Latent States (with Residuals)",
    template="plotly_white",
    showlegend=True
)

fig_multivariate.update_xaxes(title_text="Time", row=total_rows, col=1)
fig_multivariate.update_yaxes(title_text="Value", row=total_rows, col=1)

fig_multivariate.show()

## US10Y and ILS5Y5Y: Nominal and Inflation Markets

In [ ]:
df_2 = data_2.resample('W-FRI').last()['2019-01-01':].ffill().bfill()

df_2["dBrent"] = np.log(df_2["Brent"]).diff()
df_2['dSPX'] = np.log(df_2['SPX']).diff()
df_2['dDXY'] = np.log(df_2['DXY']).diff()

# Drop the first row if using diffs
df_2 = df_2.iloc[1:]
y_multivariate_2 = df_2[["US10Y", 'US ILS5Y5Y']].values

X_multivariate_2 = np.column_stack([
    df_2["dBrent"].values, 
    df_2['dSPX'].values,
    df_2['dDXY'].values            
])


In [ ]:
dcs_results_us = estimate_and_filter_gas(
    y=y_multivariate_2,
    X=X_multivariate_2,
    X_shared=True,
    phi_type='diagonal',
    kappa_type='full',
    fix_nu= 10,
    verbose=True,
    gtol=1e-5,
    estimate_omega=True,
)

mu_dcs_multivariate = dcs_results_us['mu_filtered']
beta_dcs_multivariate = dcs_results_us['beta']

N = y_multivariate_2.shape[1]
y_hat_dcs_multivariate_2 = mu_dcs_multivariate + X_multivariate_2 @ dcs_results_us['beta']

Estimated omega:
 [2.66311128 2.19269834]
Estimated Phi:
 [[0.995     0.       ]
 [0.        0.9879483]]
Estimated beta:
 [[ 0.36243571  0.04943529]
 [-0.08644802  0.33277764]
 [ 1.63780506  0.12914502]]
Estimated Omega (from lambda):
 [[0.01294062 0.        ]
 [0.         0.003657  ]]
Estimated Kappa:
 [[0.01738138 0.00096056]
 [0.0010123  0.00351455]]
Estimated nu:
 10
Log-likelihood: 663.4588804271941
AIC: -1294.9177608543882
BIC: -1222.3771394474547


In [91]:
dcs_results_us_2 = estimate_and_filter_gas(
    y=y_multivariate_2,
    X=X_multivariate_2,
    X_shared=True,
    phi_type='upper',
    kappa_type='full',
    fix_nu= 10,
    verbose=True,
    # gtol=1e-5,
    estimate_omega=True,
)


mu_dcs_multivariate_2 = dcs_results_us_2['mu_filtered']
beta_dcs_multivariate_2 = dcs_results_us_2['beta']

N = y_multivariate_2.shape[1]
y_hat_dcs_multivariate_2 = mu_dcs_multivariate_2 + X_multivariate_2 @ dcs_results_us_2['beta']

Estimated omega:
 [2.64770961 2.24187428]
Estimated Phi:
 [[0.98773193 0.08785301]
 [0.         0.99113824]]
Estimated beta:
 [[ 0.37109236  0.0579788 ]
 [-0.08188888  0.33442376]
 [ 1.61724766  0.13446736]]
Estimated Omega (from lambda):
 [[0.01258007 0.        ]
 [0.         0.00363314]]
Estimated Kappa:
 [[0.01651672 0.00097345]
 [0.00101275 0.00349757]]
Estimated nu:
 10
Log-likelihood: 668.588900069966
AIC: -1303.177800139932
BIC: -1226.1033898950652


In [105]:
kf_results_us = multivariate_KF_with_estimation(
    y = y_multivariate_2,
    X = X_multivariate_2,
    X_shared=True,
    estimate_omega=True,
)

mu_kf_multivariate_2 = kf_results_us['mu_filtered']
beta_kf_multivariate_2 = kf_results_us['beta']

N = y_multivariate_2.shape[1]
y_hat_kf_multivariate_2 = np.zeros_like(mu_kf_multivariate_2)
y_hat_kf_multivariate_2 = mu_kf_multivariate_2 + X_multivariate_2 @ kf_results_us['beta']

✅ Kalman estimation completed successfully.
Estimated parameters:
omega (unconditional mean): [0.08503273 2.02431154]
Phi (persistence matrix): 
[[9.87902156 0.95010149]
 [0.04529132 9.62590621]]
beta: 
[[ 0.1152235   0.0471599 ]
 [ 0.15219626  0.53711107]
 [ 1.33727979 -0.44129013]]
Q (state noise covariance): 
[[0.01364313 0.00277773]
 [0.00277773 0.00375802]]
R (observation noise covariance): 
[[9.90165979e-04 6.03276803e-05]
 [6.03276803e-05 8.34041868e-04]]
Log-likelihood: 652.574995147775
AIC: -1269.14999029555
BIC: -1187.5417912127498


In [106]:
# # --- Parameters ---
# burn_in_prop = 0.1  # Proportion of sample to discard for initialization
# T_multivariate_2, R_multivariate_2 = y_multivariate_2.shape
# burn_in = int(np.floor(burn_in_prop * T_multivariate_2))
# time_axis = df_2.index  # or use np.arange(T_multivariate_2) if not a DataFrame

# # --- Burn-in: slice all data arrays ---
# time_axis_plot = time_axis[burn_in:]
# y_multivariate_2_plot = y_multivariate_2[burn_in:]
# y_hat_dcs_multivariate_2_plot = y_hat_dcs_multivariate_2[burn_in:]
# y_hat_kf_multivariate_2_plot = y_hat_kf_multivariate_2[burn_in:]
# residuals_dcs_2_plot = y_multivariate_2_plot - y_hat_dcs_multivariate_2_plot
# residuals_kf_2_plot = y_multivariate_2_plot - y_hat_kf_multivariate_2_plot

# # --- Subplot layout: R rows for signals, R for residuals ---
# total_rows = 2 * R_multivariate_2
# row_heights = [0.5 / R_multivariate_2] * R_multivariate_2 + [0.5 / R_multivariate_2] * R_multivariate_2
# subplot_titles = [f"Dimension {i+1}" for i in range(R_multivariate_2)] + [f"Residuals: Dim {i+1}" for i in range(R_multivariate_2)]

# fig_multivariate_2 = make_subplots(
#     rows=total_rows, cols=1,
#     shared_xaxes=True,
#     subplot_titles=subplot_titles,
#     row_heights=row_heights
# )

# # --- Plot observed and filtered signals ---
# for i in range(R_multivariate_2):
#     fig_multivariate_2.add_trace(go.Scatter(
#         x=time_axis_plot, y=y_multivariate_2_plot[:, i],
#         mode='lines', name=f'y[{i+1}] (Observed)',
#         line=dict(color='gray', width=1.5, dash='dot'),
#         showlegend=(i==0)
#     ), row=i+1, col=1)

#     fig_multivariate_2.add_trace(go.Scatter(
#         x=time_axis_plot, y=y_hat_dcs_multivariate_2_plot[:, i],
#         mode='lines', name=f'y_hat_dcs[{i+1}]',
#         line=dict(color='orange', width=2, dash='dash'),
#         showlegend=(i==0)
#     ), row=i+1, col=1)

#     fig_multivariate_2.add_trace(go.Scatter(
#         x=time_axis_plot, y=y_hat_kf_multivariate_2_plot[:, i],
#         mode='lines', name=f'y_hat_kf[{i+1}]',
#         line=dict(color='blue', width=2, dash='dash'),
#         showlegend=(i==0)
#     ), row=i+1, col=1)

# # --- Plot residuals ---
# for i in range(R_multivariate_2):
#     fig_multivariate_2.add_trace(go.Scatter(
#         x=time_axis_plot, y=residuals_dcs_2_plot[:, i],
#         mode='lines', name=f"Residual (DCS) [{i+1}]",
#         line=dict(color='red'),
#         showlegend=False
#     ), row=R_multivariate_2 + i + 1, col=1)

#     fig_multivariate_2.add_trace(go.Scatter(
#         x=time_axis_plot, y=residuals_kf_2_plot[:, i],
#         mode='lines', name=f"Residual (Kalman) [{i+1}]",
#         line=dict(color='blue', dash='dot'),
#         showlegend=False
#     ), row=R_multivariate_2 + i + 1, col=1)

# # --- Final layout tweaks ---
# fig_multivariate_2.update_layout(
#     height=300 * R_multivariate_2 + 200,
#     width=950,
#     title_text="dcs Filter vs Kalman Filter: Signals and Residuals (After Burn-in)",
#     template="plotly_white",
#     showlegend=True
# )

# fig_multivariate_2.update_xaxes(title_text="Time", row=total_rows, col=1)
# fig_multivariate_2.update_yaxes(title_text="Value", row=total_rows, col=1)

# fig_multivariate_2.show()


In [109]:
T_multivariate_2, R_multivariate_2 =  y_multivariate_2.shape
time_axis = df.index #list(range(T_multivariate_2))

# --- Residuals ---
residuals_dcs_2 = y_multivariate_2 - y_hat_dcs_multivariate_2
residuals_kf_2 = y_multivariate_2 - y_hat_kf_multivariate_2


total_rows = 2 * R_multivariate_2
row_heights = [0.5 / R_multivariate_2] * R_multivariate_2 + [0.5 / R_multivariate_2] * R_multivariate_2


fig_multivariate_2 = make_subplots(
    rows=total_rows, cols=1,
    shared_xaxes=True,
    subplot_titles=[f"Dimension {i+1}" for i in range(R_multivariate_2)] + ["Residuals: DCS vs Kalman"],
    row_heights=row_heights
)

# Plot Observed and Filtered 
for i in range(R_multivariate_2):
    fig_multivariate_2.add_trace(go.Scatter(x=time_axis, y=y_multivariate_2[:, i],
        mode='lines', name=f'y[{i+1}] (Observed)',
        line=dict(color='gray', width=1.5, dash='dot')), row=i+1, col=1)

    fig_multivariate_2.add_trace(go.Scatter(x=time_axis, y=y_hat_dcs_multivariate_2[:, i],
        mode='lines', name=f'y_hat_dcs[{i+1}]',
        line=dict(color='orange', width=2, dash='dash')), row=i+1, col=1)

    fig_multivariate_2.add_trace(go.Scatter(x=time_axis, y=y_hat_kf_multivariate_2[:, i],
        mode='lines', name=f'y_hat_kf[{i+1}]',
        line=dict(color='blue', width=2, dash='dash')), row=i+1, col=1)


# Plot the residuals
for i in range(R_multivariate_2):

    # Add residuals to row=i+1+R
    fig_multivariate_2.add_trace(go.Scatter(
        x=time_axis, y=residuals_dcs_2[:, i], name=f"Residual (DCS) [{i+1}]",
        line=dict(color='red')), row=i + 1 + R_multivariate_2, col=1)

    fig_multivariate_2.add_trace(go.Scatter(
        x=time_axis, y=residuals_kf_2[:, i], name=f"Residual (Kalman) [{i+1}]",
        line=dict(color='blue', dash='dot')), row=i + 1 + R_multivariate_2, col=1)

# --- Final layout ---
fig_multivariate_2.update_layout(
    height=300*R_multivariate_2 + 200, width=950,
    title_text="mDCS-t Filter: Filtered Latent States (with Residuals)",
    template="plotly_white",
    showlegend=True
)

fig_multivariate_2.update_xaxes(title_text="Time", row=total_rows, col=1)
fig_multivariate_2.update_yaxes(title_text="Value", row=total_rows, col=1)

fig_multivariate_2.show()